In [6]:
#Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
%cd /content/gdrive/MyDrive/Deepfake_detector/mini_df_data/train_dataset
# %ls
# %cd gdrive
# %ls


/content/gdrive/MyDrive/Deepfake_detector/mini_df_data/train_dataset


In [8]:
#To get the average frame count
import json
import glob
import numpy as np
import cv2
import copy
#change the path accordingly
video_files =  glob.glob('fake/*.mp4')
#video_files1 =  glob.glob('/content/dfdc_train_part_0/*.mp4')
#video_files += video_files1
frame_count = []
for video_file in video_files:
  cap = cv2.VideoCapture(video_file)
  if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<150):
    video_files.remove(video_file)
    continue
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("frames" , frame_count)
print("Total number of videos: " , len(frame_count))
print('Average frame per video:',np.mean(frame_count))

frames [300, 300, 300, 301, 300, 301, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 301, 300, 300, 300, 300, 300, 301, 300, 300, 300, 300, 167, 300, 167, 300, 300, 301, 301, 301, 301, 301, 300, 300, 301, 300, 300]
Total number of videos:  44
Average frame per video: 294.1818181818182


In [9]:
# # to extract frame
# def frame_extract(path):
#   vidObj = cv2.VideoCapture(path)
#   success = 1
#   while success:
#       success, image = vidObj.read()
#       if success:
#           yield image
!pip3 install face_recognition
# # !mkdir 'content/gdrive/MyDrive/Deepfake_detector/dataset/processed_dataset/fake'
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition
from tqdm.autonotebook import tqdm
# # process the frames
# def create_face_videos(path_list,out_dir):
#   already_present_count =  glob.glob(out_dir+'*.mp4')
#   print("No of videos already present " , len(already_present_count))
#   for path in tqdm(path_list):
#     out_path = os.path.join(out_dir,path.split('/')[-1])
#     file_exists = glob.glob(out_path)
#     if(len(file_exists) != 0):
#       print("File Already exists: " , out_path)
#       continue
#     frames = []
#     flag = 0
#     face_all = []
#     frames1 = []
#     out = cv2.VideoWriter(out_path,cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112,112))
#     for idx,frame in enumerate(frame_extract(path)):
#       #if(idx % 3 == 0):
#       if(idx <= 150):
#         frames.append(frame)
#         if(len(frames) == 4):
#           faces = face_recognition.batch_face_locations(frames)
#           for i,face in enumerate(faces):
#             if(len(face) != 0):
#               top,right,bottom,left = face[0]
#             try:
#               out.write(cv2.resize(frames[i][top:bottom,left:right,:],(112,112)))
#             except:
#               pass
#           frames = []
#     try:
#       del top,right,bottom,left
#     except:
#       pass
#     out.release()
import gc

def create_face_videos(path_list, out_dir):
    already_present_count = len(glob.glob(os.path.join(out_dir, '*.mp4')))
    print("Number of videos already present:", already_present_count)

    for path in tqdm(path_list):
        out_path = os.path.join(out_dir, os.path.basename(path))
        if os.path.exists(out_path):
            print("File already exists:", out_path)
            continue

        vidObj = cv2.VideoCapture(path)
        out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112,112))
        frames = []

        try:
            success, frame = vidObj.read()
            while success and len(frames) <= 150:
                frames.append(frame)
                if len(frames) == 4:
                    # Reduce batch size to manage memory usage
                    faces = face_recognition.batch_face_locations(frames, number_of_times_to_upsample=0, batch_size=2)
                    for i, face_locations in enumerate(faces):
                        for top, right, bottom, left in face_locations:
                            cropped_frame = cv2.resize(frame[top:bottom, left:right], (112, 112))
                            out.write(cropped_frame)
                    frames.clear()  # Clear frames list to free up memory
                success, frame = vidObj.read()
        except Exception as e:
            print(f"Error processing video {path}: {str(e)}")

        out.release()
        vidObj.release()
        cv2.destroyAllWindows()
        gc.collect()  # Explicitly call garbage collector after processing each video



In [13]:
create_face_videos(video_files,'/content/gdrive/MyDrive/Deepfake_detector/dataset/processed_dataset/fake')

Number of videos already present: 2


  0%|          | 0/45 [00:00<?, ?it/s]

File already exists: /content/gdrive/MyDrive/Deepfake_detector/dataset/processed_dataset/fake/4958.mp4
File already exists: /content/gdrive/MyDrive/Deepfake_detector/dataset/processed_dataset/fake/4957.mp4
